# Artists present per month

The objective here for me is to see the persistence of artists over the course of months in the dataset, with a `start_date` and `end_date` set and a `padding`

In [1]:
from tqdm.notebook import tqdm
from collections import OrderedDict
import pandas as pd
import datetime, maya, pytz, re, json, copy

In [2]:
start_date = datetime.datetime(year=1929, month=1, day=1)
end_date = datetime.datetime(year=1941, month=1, day=1)
padding = datetime.timedelta(weeks=12)


# Fix timezones
timezone = pytz.timezone("UTC")
start_date = timezone.localize(start_date)
end_date = timezone.localize(end_date)

In [3]:
def get_clean_df(only_performer_and_venue=True):
    def get_performer(row):
        if row['Normalized performer'] != '':
            return row['Normalized performer']
        if row['Performer'] != '':
            return row['Performer']
        if row['Performer first-name'] != '' and row['Performer last-name'] != '':
            return row['Performer first-name'] + ' ' + row['Performer last-name']
        return ''

    def get_revue(row):
        if row['Normalized Revue Name'] != '':
            return row['Normalized Revue Name']
        if row['Revue name'] != '':
            return row['Revue name']
        return ''

    def get_venue(row, get_unique_venue=True):
        if get_unique_venue:
            if row['Unique venue'] != '':
                return row['Unique venue']

        if row['Normalized Venue'] != '':
            return row['Normalized Venue']
        if row['Venue'] != '':
            return row['Venue']
        return ''

    def get_city(row):
        if row['Normalized City'] != '':
            return row['Normalized City']
        if row['City'] != '':
            return row['City']
        return ''

    def get_source(row):
        if row['Source clean'] != '':
            return row['Source clean']
        if row['Source'] != '':
            return row['Source']
        return ''

    import maya
    from maya import pendulum

    def test_date(row, accept_only_full_dates=True):
        if accept_only_full_dates and not re.search(r'\d{4}\-\d{2}\-\d{2}', row["Date"]):
            return ''
        try:
            m = maya.parse(row["Date"]).datetime()
            return m
        except pendulum.exceptions.ParserError:
            return ''
        except ValueError:
            return ''

    drop_columns = ['Legal name', 'Alleged age', 'Assumed birth year', 'EIMA', 'Search (newspapers.com)', 'Search (fulton)', 'Imported from former archive', 'Edge Comment', 'Exclude from visualization', 'Comment on node: performer', 'Comment on node: venue', 'Comment on node: city', 'Comment on edge: revue', 'Blackface', 'Sepia', 'Fan dancer/Sally Rand', 'Exotic/erotic/oriental dancer/Gypsy', 'Has image', 'Address', 'Vaudeville Circuit/Circus', 'Unsure whether drag artist']

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT0E0Y7txIa2pfBuusA1cd8X5OVhQ_D0qZC8D40KhTU3xB7McsPR2kuB7GH6ncmNT3nfjEYGbscOPp0/pub?gid=0&single=true&output=csv')

    df = df.replace('—', '')
    df = df.fillna('')


    # Set the dates
    df['Date'] = df.apply(lambda row: test_date(row), axis=1)

    # No required date
    no_date = df[(df['Date'] == '')].index
    df = df.drop(no_date)

    # Excluded
    excluded = df[df['Exclude from visualization'] == True].index
    df = df.drop(excluded)

    # Excluded
    unsure_drag = df[df['Unsure whether drag artist'] == True].index
    df = df.drop(unsure_drag)

    # Drop unnecessary columns
    df = df.drop(['Category', 'Normalized City', 'City'], axis='columns')

    # Fix performers
    df['Performer'] = df.apply(lambda row: get_performer(row), axis=1)
    df = df.drop(['Normalized performer', 'Performer first-name', 'Performer last-name'], axis='columns')

    # Fix venue
    df['Venue'] = df.apply(lambda row: get_venue(row), axis=1)
    df = df.drop(['Normalized Venue', 'Unique venue'], axis='columns')

    # Fix city *since we're using "Unique venue", we don't need city
    # df['City'] = df.apply(lambda row: get_city(row), axis=1)
    # df = df.drop(['Normalized City'], axis='columns')

    # Fix revue
    df['Revue'] = df.apply(lambda row: get_revue(row), axis=1)
    df = df.drop(['Revue name', 'Normalized Revue Name'], axis='columns')

    # Fix source
    df['Source'] = df.apply(lambda row: get_source(row), axis=1)
    df = df.drop(['Source clean'], axis='columns')

    # drop rows with none of the required data
    no_data = df[(df['Performer'] == '') & (df['Venue'] == '')].index 
    df = df.drop(no_data)

    if only_performer_and_venue:
        # drop rows with none of the required data
        no_data = df[(df['Venue'] == '') | (df['Performer'] == '')].index 
        df = df.drop(no_data)

    # Drop rest of columns
    df = df.drop(drop_columns, axis='columns')
    
    return df

In [4]:
df = get_clean_df()


# FILTER

def find_unnamed(row):
    if 'unnamed' in row.Performer.lower():
        return True
    return False

# Filtering by date
filtered_df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)].copy()

# Filter out unnamed performers
filtered_df['Unnamed'] = filtered_df.apply(lambda row: find_unnamed(row), axis=1)
unnamed = filtered_df[filtered_df['Unnamed'] == True].index
filtered_df = filtered_df.drop(unnamed)

# Remove duplicates of subset of Date - Performer - Venue
duplicated = filtered_df[filtered_df.duplicated(subset=['Date', 'Performer', 'Venue'], keep='first')].index
filtered_df.drop(duplicated)

# Reset index
filtered_df = filtered_df.reset_index()
filtered_df = filtered_df.drop(['index'], axis='columns')

# Sort by performer
filtered_df = filtered_df.sort_values('Performer')

In [5]:
def get_check(year, month, tolerance=2, min_year=1930, max_year=1940):
    ''' Returns the padding around a given `year` and `month` within a span (set by `min_year` and `max_year`) and with the padding set to `tolerance`. '''
    
    all_dates = []

    years = [x for x in range(min_year-2, max_year+2)]
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    for _year in years:
        for _month in months:
            all_dates.append((_year, _month))
            
    ix = all_dates.index((year, month))
    
    dates = [all_dates[ix + x] for x in range(-tolerance, tolerance+1)]
    
    check_years = []
    check_months = []
    for year, month in dates:
        check_years.append(year)
        check_months.append(month)
    
    return check_years, check_months

In [6]:
# Set up all the variables we need
month_list = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
paddings = [0, 1, 2, 3, 4, 5]
calendars = {}
padded_datasets = {}

In [7]:
# Get all years in dataset in order
years_in_dataset = list(set([x.year for x, _ in filtered_df.sort_values('Date').groupby('Date')]))

# Add them to calendar
calendar = OrderedDict({year: [x for x in month_list if x] for year in years_in_dataset})

# `calendar` is now an empty dictionary that contains all of the years (with concomitant months) for the dataset
print(calendar)

OrderedDict([(1929, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1930, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1931, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1932, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1933, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1934, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1935, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1936, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1937, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1938, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), (1939, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun

In [8]:
# We have to re-organize `calendar` to be a dict of `years: months: [list of performers]`:
_calendar = {}
for y, v in calendar.items():
    _calendar[y] = {}
    for m in [m for m in v if m]:
        _calendar[y][m] = []
calendar = _calendar

# save as `empty_calendar` for later use
empty_calendar = copy.deepcopy(calendar)

# `calendar now looks like we want it to`
print('years:', calendar.keys())
print('months (for each year):', calendar[1930].keys())

years: dict_keys([1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940])
months (for each year): dict_keys(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])


In [9]:
# Loop through a list of all the performers
for performer, data in filtered_df.groupby('Performer'):
    all_dates = data.sort_values('Date').Date
    # Zip the years and months together
    dates = zip([x.year for x in all_dates], [month_list[x.month] for x in all_dates])

    # Loop through them and add their name to the lists under each [year][month] in calendar
    for year, month in dates:
        calendar[year][month].append(performer)
        calendar[year][month] = list(set(calendar[year][month]))
        
# `calendar` now has a list of unique performers in the dataset who appear for yeach [year][month]
print(calendar[1930])

{'Jan': ['Francis Renault', 'G. Doran', 'Frank Doran', 'Arthur G. West', 'Adrian Ames'], 'Feb': ['George Denny', 'Francis Renault', 'G. Doran', 'Jean Malin', 'Frank Doran', 'Arthur G. West', 'Cliff LaVerne', 'Lynn Overman', 'Frances Dunn'], 'Mar': ['Francis Renault', 'Wallie Groff', 'Frank Doran', 'G. Doran', '— Brennan', 'Nine Collegians', 'Arthur G. West', 'Clara Bow', 'E. Walter', 'Mack Youge'], 'Apr': ['Jarahal', 'Francis Renault', 'Jean Malin', 'Stanley Rogers'], 'May': ['Francis Renault', 'Francois Densmore'], 'Jun': ['Tommy Mann', 'Francis Renault', 'Tom Martelle', 'Karyl Norman'], 'Jul': ['Francis Renault', 'Karyl Norman'], 'Aug': ['Jarahal', 'Francis Renault'], 'Sep': ['Jackie Maye', 'Francis Renault', 'G. Doran', 'Frank Doran', 'Jean Malin', 'Tex Hendricks', 'Frances Fay', 'Arthur G. West', 'La Belle Rose', 'Frances Dunn', 'Arthur Budd', 'Jarahal'], 'Oct': ['Vincent McFarland', 'Jackie Maye', 'Ramon Strobeck', 'Francis Renault', 'Jean Malin', 'Cecil Mason', 'Jerry Sullivan', 

In [10]:
calendar_checks = {}
for year, months in {year: [x for x in months.keys()] for year, months in calendar.items()}.items():
    calendar_checks[year] = {}
    for month in months:
        calendar_checks[year][month] = {padding: get_check(year, month, tolerance=padding, min_year=min(years_in_dataset), max_year=max(years_in_dataset)) for padding in paddings}
            
# `calendar_checks` now has the same calendar structure as the other `calendar` except each [year][month] value
# follows the strcuture {padding: check}
calendar_checks[1939]

{'Jan': {0: ([1939], ['Jan']),
  1: ([1938, 1939, 1939], ['Dec', 'Jan', 'Feb']),
  2: ([1938, 1938, 1939, 1939, 1939], ['Nov', 'Dec', 'Jan', 'Feb', 'Mar']),
  3: ([1938, 1938, 1938, 1939, 1939, 1939, 1939],
   ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr']),
  4: ([1938, 1938, 1938, 1938, 1939, 1939, 1939, 1939, 1939],
   ['Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May']),
  5: ([1938, 1938, 1938, 1938, 1938, 1939, 1939, 1939, 1939, 1939, 1939],
   ['Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec',
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun'])},
 'Feb': {0: ([1939], ['Feb']),
  1: ([1939, 1939, 1939], ['Jan', 'Feb', 'Mar']),
  2: ([1938, 1939, 1939, 1939, 1939], ['Dec', 'Jan', 'Feb', 'Mar', 'Apr']),
  3: ([1938, 1938, 1939, 1939, 1939, 1939, 1939],
   ['Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May']),
  4: ([1938, 1938, 1938, 1939, 1939, 1939, 1939, 1939, 1939],
   ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']),
  5: ([1938, 193

In [11]:
for padding in [0, 1, 2, 3, 4, 5]:
    # set up a `padded_dataset` from the empty calendar from before
    padded_dataset = copy.deepcopy(empty_calendar)
    
    for year in years_in_dataset:
        for month in [x for x in month_list if x]:
            # print(year, month)
            artists = calendar[year][month] # get the artists that appeared in the dataset for that specific month

            # they should be spread out, depending on the padding — by 0 months, 1 month, 2 months, and 3 months)
            spread_years, spread_months = calendar_checks[year][month][padding]
            
            print()
            print(f'currently we are looking at {month} {year} when there were {len(artists)} active performers.')
            print(f'This month should spread to {", ".join(" ".join(x) for x in zip(spread_months, [str(x) for x in spread_years]))}.')
            
            for y, m in zip(spread_years, spread_months):
                try:
                    padded_dataset[y][m].extend(artists)
                    padded_dataset[y][m] = list(set(padded_dataset[y][m]))
                except KeyError as k:
                    pass # print(k, 'does not exist in padded_dataset') # This will occur as the padding means that we overflow into years outside of the purview of this visualization
                
    padded_datasets[padding] = padded_dataset.copy()


currently we are looking at Jan 1929 when there were 2 active performers.
This month should spread to Jan 1929.

currently we are looking at Feb 1929 when there were 1 active performers.
This month should spread to Feb 1929.

currently we are looking at Mar 1929 when there were 2 active performers.
This month should spread to Mar 1929.

currently we are looking at Apr 1929 when there were 1 active performers.
This month should spread to Apr 1929.

currently we are looking at May 1929 when there were 2 active performers.
This month should spread to May 1929.

currently we are looking at Jun 1929 when there were 0 active performers.
This month should spread to Jun 1929.

currently we are looking at Jul 1929 when there were 1 active performers.
This month should spread to Jul 1929.

currently we are looking at Aug 1929 when there were 1 active performers.
This month should spread to Aug 1929.

currently we are looking at Sep 1929 when there were 1 active performers.
This month should spr

In [12]:
def save_as_json(dict, path):
    with open(path, 'w+') as fp:
        json.dump(fp=fp, obj=dict)
        
    return True

In [13]:
for padding in padded_datasets:
    save_as_json(padded_datasets[padding], f'../../drag-data-browser/docs/data/continuous-performances-padding-{padding}-detail.json')
    save_as_json(padded_datasets[padding], f'../../drag-data-1930s/network-app/data/continuous-performances-padding-{padding}-detail.json')

In [14]:
def save_as_json(df, path):
    rows = []
    for ix,row in df.iterrows():
        rows.append({
            'date': ix.strftime('%Y-%m-%d'),
            'month': row.month,
            'num_artists': row.num_artists,
            'year': row.year
        })
        
    with open(path, 'w+') as fp:
        json.dump(fp=fp, obj=rows)
        
    return True


for padding in padded_datasets:
    df_summary = pd.DataFrame()

    for year, dp in padded_datasets[padding].items():
        for month, performers in dp.items():
            d = {'month': month, 'year': year, 'num_artists': len(performers)}
            s = pd.Series(d, name=pd.to_datetime(f'{year}-{month}'))
            df_summary = df_summary.append(s)
    df_summary['num_artists'] = df_summary['num_artists'].astype(int)
    df_summary['year'] = df_summary['year'].astype(int)


    save_as_json(df_summary, f'../../drag-data-browser/docs/data/continuous-performances-padding-{padding}.json')
    save_as_json(df_summary, f'../../drag-data-1930s/network-app/data/continuous-performances-padding-{padding}.json')
